# Week 5: Exploratory Data Analysis (EDA) Fundamentals

**Goal:** Master exploratory data analysis techniques to uncover insights in marketing data.

**Time Commitment:** ~1 hour per day × 7 days = 7 hours total

**What You'll Learn:**
- Descriptive statistics and summary measures
- Distribution analysis and outlier detection
- Correlation analysis between metrics
- Data profiling and quality assessment
- Segmentation analysis
- Cohort analysis
- End-to-end campaign EDA project

**Why This Matters:**
As a Marketing Measurement Partner, EDA helps you:
- Understand data quality issues before analysis
- Discover patterns and anomalies
- Identify which metrics matter most
- Segment audiences effectively
- Find optimization opportunities
- Validate assumptions

EDA is the foundation of data-driven decision making. It's what you do BEFORE building models or dashboards.

---

## Setup: Load Marketing Dataset

We'll work with a comprehensive marketing dataset for this week.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully!")

In [ ]:
# Generate realistic campaign performance data
np.random.seed(42)

# Create 90 days of data for 10 campaigns
dates = pd.date_range('2024-10-01', periods=90, freq='D')
campaigns = [
    'Search_Brand', 'Search_Generic', 'FB_Prospecting', 'FB_Retargeting',
    'Display_Remarketing', 'Instagram_Stories', 'LinkedIn_B2B', 
    'TikTok_Awareness', 'YouTube_Video', 'Search_Competitor'
]
channels = ['Google', 'Google', 'Meta', 'Meta', 'Google', 'Meta', 'LinkedIn', 'TikTok', 'Google', 'Google']
campaign_types = ['Search', 'Search', 'Social', 'Social', 'Display', 'Social', 'Social', 'Video', 'Video', 'Search']

data = []
for i, campaign in enumerate(campaigns):
    for date in dates:
        # Add day-of-week seasonality
        dow_multiplier = 1.3 if date.dayofweek < 5 else 0.7  # Weekday boost
        
        # Different campaigns have different base performance
        if campaign_types[i] == 'Search':
            impressions = int(np.random.normal(5000, 1000) * dow_multiplier)
            ctr = np.random.normal(0.05, 0.01)
            cvr = np.random.normal(0.04, 0.008)
        elif campaign_types[i] == 'Social':
            impressions = int(np.random.normal(25000, 5000) * dow_multiplier)
            ctr = np.random.normal(0.03, 0.008)
            cvr = np.random.normal(0.025, 0.006)
        elif campaign_types[i] == 'Display':
            impressions = int(np.random.normal(40000, 8000) * dow_multiplier)
            ctr = np.random.normal(0.015, 0.004)
            cvr = np.random.normal(0.02, 0.005)
        else:  # Video
            impressions = int(np.random.normal(50000, 10000) * dow_multiplier)
            ctr = np.random.normal(0.02, 0.005)
            cvr = np.random.normal(0.015, 0.004)
        
        clicks = int(impressions * max(0.001, ctr))
        conversions = int(clicks * max(0.001, cvr))
        cpc = np.random.uniform(0.5, 3.0) if campaign_types[i] == 'Search' else np.random.uniform(0.3, 1.5)
        cost = clicks * cpc
        aov = np.random.uniform(80, 200)  # Average order value
        revenue = conversions * aov
        
        data.append({
            'date': date,
            'campaign': campaign,
            'channel': channels[i],
            'campaign_type': campaign_types[i],
            'impressions': max(0, impressions),
            'clicks': max(0, clicks),
            'conversions': max(0, conversions),
            'cost': max(0, cost),
            'revenue': max(0, revenue)
        })

df = pd.DataFrame(data)

# Calculate metrics
df['ctr'] = df['clicks'] / df['impressions']
df['cvr'] = df['conversions'] / df['clicks'].replace(0, np.nan)
df['cpa'] = df['cost'] / df['conversions'].replace(0, np.nan)
df['roas'] = df['revenue'] / df['cost'].replace(0, np.nan)
df['cpc'] = df['cost'] / df['clicks'].replace(0, np.nan)

print(f"✅ Dataset created: {len(df)} rows")
print(f"\nDate range: {df['date'].min()} to {df['date'].max()}")
print(f"Campaigns: {df['campaign'].nunique()}")
print(f"\nFirst few rows:")
df.head(10)

## 📅 Day 29: Descriptive Statistics (~60 min)

### Learning Objectives
- Calculate measures of central tendency (mean, median, mode)
- Understand measures of spread (std, variance, range, IQR)
- Use pandas describe() effectively
- Calculate statistics by groups

### The Business Problem
Before diving into analysis, you need to understand your data:
- What's the typical daily performance?
- How much does performance vary?
- Are there campaigns with unusual patterns?

### 📖 Concept: Central Tendency

Central tendency tells you what's "typical."

In [ ]:
# Basic statistics for cost
print("Daily Cost Statistics:")
print(f"Mean (average): ${df['cost'].mean():.2f}")
print(f"Median (50th percentile): ${df['cost'].median():.2f}")
print(f"Mode (most common): ${df['cost'].mode().values[0]:.2f}")
print(f"\nMin: ${df['cost'].min():.2f}")
print(f"Max: ${df['cost'].max():.2f}")
print(f"Range: ${df['cost'].max() - df['cost'].min():.2f}")

### 📖 Concept: Spread/Variability

Variability tells you how consistent the data is.

In [ ]:
# Measures of spread
print("ROAS Variability:")
print(f"Standard Deviation: {df['roas'].std():.2f}")
print(f"Variance: {df['roas'].var():.2f}")
print(f"Coefficient of Variation: {df['roas'].std() / df['roas'].mean():.2%}")
print(f"\nInterquartile Range (IQR):")
print(f"25th percentile (Q1): {df['roas'].quantile(0.25):.2f}")
print(f"50th percentile (Q2/median): {df['roas'].quantile(0.50):.2f}")
print(f"75th percentile (Q3): {df['roas'].quantile(0.75):.2f}")
print(f"IQR (Q3 - Q1): {df['roas'].quantile(0.75) - df['roas'].quantile(0.25):.2f}")

### 📖 Concept: The describe() Method

Get all statistics at once.

In [ ]:
# Comprehensive statistical summary
print("Campaign Performance Summary Statistics:")
df[['impressions', 'clicks', 'conversions', 'cost', 'revenue', 'ctr', 'cvr', 'cpa', 'roas']].describe()

### 📖 Concept: Grouped Statistics

Compare statistics across categories.

In [ ]:
# Statistics by channel
print("Performance by Channel:")
channel_stats = df.groupby('channel').agg({
    'cost': ['mean', 'median', 'std'],
    'roas': ['mean', 'median', 'std'],
    'conversions': ['mean', 'median', 'std']
}).round(2)

print(channel_stats)

In [ ]:
# Statistics by campaign type
print("\nPerformance by Campaign Type:")
type_stats = df.groupby('campaign_type')[['ctr', 'cvr', 'cpa', 'roas']].agg(['mean', 'median', 'std']).round(4)
print(type_stats)

### ✏️ Exercise 1: Analyze Campaign Consistency

In [ ]:
# YOUR CODE HERE
# For each campaign, calculate:
# 1. Mean daily ROAS
# 2. Standard deviation of daily ROAS
# 3. Coefficient of variation (std/mean)
# 4. Sort by coefficient of variation to find most/least consistent campaigns
#
# Which campaign is most consistent? Which is most volatile?



### 🎯 Day 29 Mini-Project: Campaign Performance Profile

Create a comprehensive statistical profile for each campaign.

In [ ]:
# YOUR CODE HERE
# Create a campaign profile DataFrame showing:
# - Campaign name
# - Total days active (count of rows)
# - Mean/median/std for: cost, conversions, ROAS
# - Best day (max ROAS) and worst day (min ROAS)
# - Consistency score (1 / coefficient_of_variation for ROAS)
#
# Sort by mean ROAS descending
# Format nicely for presentation



### 🎓 Day 29 Key Takeaways

✅ Mean shows average, median shows middle value  
✅ Standard deviation measures consistency  
✅ IQR is robust to outliers  
✅ describe() gives quick overview  
✅ Always segment statistics by important groups  

**Next:** Tomorrow we'll analyze distributions and find outliers!

---

## 📅 Day 30: Distributions and Outliers (~60 min)

### Learning Objectives
- Understand different distribution shapes
- Identify skewness and kurtosis
- Detect outliers using IQR method
- Decide how to handle outliers

### The Business Problem
Marketing data often has:
- Extreme values (viral posts, Black Friday spikes)
- Skewed distributions (most days normal, few huge)
- Data quality issues (tracking errors)

You need to identify and handle these appropriately.

### 📖 Concept: Distribution Shape

In [ ]:
# Analyze distribution of ROAS
print("ROAS Distribution Analysis:")
print(f"Mean: {df['roas'].mean():.2f}")
print(f"Median: {df['roas'].median():.2f}")
print(f"Skewness: {df['roas'].skew():.2f}")
print(f"  (0 = symmetric, >0 = right skew, <0 = left skew)")
print(f"Kurtosis: {df['roas'].kurtosis():.2f}")
print(f"  (0 = normal, >0 = heavy tails, <0 = light tails)")

### 📖 Concept: Outlier Detection (IQR Method)

IQR method: values beyond Q1 - 1.5×IQR or Q3 + 1.5×IQR

In [ ]:
# Detect outliers in daily cost
Q1 = df['cost'].quantile(0.25)
Q3 = df['cost'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['cost'] < lower_bound) | (df['cost'] > upper_bound)]

print(f"Cost Outlier Detection:")
print(f"Q1: ${Q1:.2f}")
print(f"Q3: ${Q3:.2f}")
print(f"IQR: ${IQR:.2f}")
print(f"Lower bound: ${lower_bound:.2f}")
print(f"Upper bound: ${upper_bound:.2f}")
print(f"\nOutliers found: {len(outliers)} ({len(outliers)/len(df)*100:.1f}% of data)")
print(f"\nOutlier examples:")
print(outliers[['date', 'campaign', 'cost', 'conversions']].head())

### 📖 Concept: Z-Score Method

Alternative: flag values > 3 standard deviations from mean

In [ ]:
# Z-score outlier detection for ROAS
df['roas_zscore'] = (df['roas'] - df['roas'].mean()) / df['roas'].std()
roas_outliers = df[abs(df['roas_zscore']) > 3]

print(f"ROAS Z-Score Outliers (>3 std from mean):")
print(f"Outliers found: {len(roas_outliers)}")
print(f"\nExamples:")
print(roas_outliers[['date', 'campaign', 'roas', 'roas_zscore']].head())

### ✏️ Exercise 2: Multi-Metric Outlier Analysis

In [ ]:
# YOUR CODE HERE
# Create a function to detect outliers using IQR method
# Apply it to: CTR, CVR, CPA, ROAS
# For each metric:
# 1. Count outliers
# 2. Show % of data that are outliers
# 3. Identify which campaigns have most outliers
#
# Are outliers consistent across metrics or different days/campaigns?



### 🎯 Day 30 Mini-Project: Data Quality Report

Create a comprehensive data quality and outlier report.

In [ ]:
# YOUR CODE HERE
# Build a data quality dashboard showing:
# 1. Missing values by column
# 2. Outlier counts by metric
# 3. Campaigns with unusual patterns (high outlier %)
# 4. Days with unusual activity (multiple metrics outliers)
# 5. Recommendations for data cleaning
#
# Should you remove outliers? Cap them? Investigate them?



### 🎓 Day 30 Key Takeaways

✅ Skewness indicates distribution asymmetry  
✅ IQR method robust for outlier detection  
✅ Z-score method assumes normal distribution  
✅ Not all outliers are errors - investigate first!  
✅ Document outlier handling decisions  

**Next:** Tomorrow we'll explore correlations between metrics!

---

## 📅 Day 31: Correlation Analysis (~60 min)

### Learning Objectives
- Calculate correlation coefficients
- Interpret correlation strength and direction
- Create correlation matrices
- Understand correlation vs causation

### The Business Problem
Which metrics move together?
- Does higher spend lead to more conversions?
- Are CTR and CVR related?
- Which factors predict campaign success?

### 📖 Concept: Correlation Coefficient

Measures linear relationship between two variables (-1 to +1)

In [ ]:
# Correlation between cost and conversions
correlation = df['cost'].corr(df['conversions'])
print(f"Correlation between Cost and Conversions: {correlation:.3f}")
print(f"\nInterpretation:")
print(f"  +1.0 = perfect positive correlation")
print(f"  0.0 = no correlation")
print(f"  -1.0 = perfect negative correlation")
print(f"\nThis correlation of {correlation:.3f} indicates ", end="")
if abs(correlation) > 0.7:
    print("strong", end=" ")
elif abs(correlation) > 0.4:
    print("moderate", end=" ")
else:
    print("weak", end=" ")
print("positive relationship" if correlation > 0 else "negative relationship")

### 📖 Concept: Correlation Matrix

View all pairwise correlations at once

In [ ]:
# Correlation matrix for key metrics
metrics = ['impressions', 'clicks', 'conversions', 'cost', 'revenue', 'ctr', 'cvr', 'roas']
corr_matrix = df[metrics].corr()

print("Correlation Matrix:")
print(corr_matrix.round(2))

### 📖 Concept: Interpreting Correlations for Marketing

Key insights from correlation analysis

In [ ]:
# Find strongest correlations with ROAS
roas_correlations = df[metrics].corr()['roas'].sort_values(ascending=False)
print("Correlations with ROAS (sorted):")
print(roas_correlations.round(3))
print("\nInsight: Which metrics are most predictive of ROAS?")

### 📖 Concept: Correlation by Group

Correlations may differ by segment

In [ ]:
# Correlation between cost and conversions by campaign type
print("Cost vs Conversions Correlation by Campaign Type:")
for campaign_type in df['campaign_type'].unique():
    subset = df[df['campaign_type'] == campaign_type]
    corr = subset['cost'].corr(subset['conversions'])
    print(f"{campaign_type:12s}: {corr:.3f}")

### ✏️ Exercise 3: Correlation Analysis

In [ ]:
# YOUR CODE HERE
# 1. Calculate correlation between CTR and CVR
# 2. Are they positively or negatively correlated?
# 3. Calculate this correlation separately for each channel
# 4. Does the relationship differ by channel?
#
# What does this tell you about campaign optimization?



### 🎯 Day 31 Mini-Project: Correlation-Based Insights

Use correlation analysis to derive business insights.

In [ ]:
# YOUR CODE HERE
# Create a correlation-based insight report:
# 
# 1. Identify metrics with strongest positive correlation to revenue
# 2. Identify metrics with strongest negative correlation to CPA
# 3. Find pairs of metrics with surprising correlations
# 4. Calculate correlations by campaign and identify outliers
#    (campaigns with very different correlation patterns)
# 5. Provide 3-5 actionable insights based on correlations
#
# Example: "Increasing CTR is strongly correlated with better ROAS in 
# Search campaigns (r=0.72) but not in Display (r=0.18)"



### 🎓 Day 31 Key Takeaways

✅ Correlation measures linear relationships  
✅ Ranges from -1 (negative) to +1 (positive)  
✅ Correlation ≠ causation!  
✅ Always segment correlations by groups  
✅ Use correlation to generate hypotheses, not conclusions  

**Next:** Tomorrow we'll learn data profiling techniques!

---

## 📅 Day 32-34: Additional EDA Topics

**Day 32: Data Profiling**
- Missing value analysis
- Cardinality assessment
- Data type validation
- Uniqueness checks

**Day 33: Segmentation Analysis**
- K-means clustering basics
- RFM analysis (Recency, Frequency, Monetary)
- Campaign performance tiers
- Customer segmentation

**Day 34: Cohort Analysis**
- Time-based cohorts
- Cohort retention analysis
- Campaign launch cohorts
- Seasonal pattern detection

---

## 📅 Day 35: Week 5 Capstone - Campaign EDA Project (~60 min)

### Project: Comprehensive Campaign Performance EDA

**Scenario:**  
You've been handed 90 days of campaign data for 10 campaigns across 4 channels. Before making optimization recommendations, you need to perform thorough exploratory analysis.

**Your Task:**
Conduct end-to-end EDA and deliver insights to stakeholders.

**Deliverables:**
1. Data quality assessment
2. Descriptive statistics summary
3. Distribution analysis and outlier investigation
4. Correlation insights
5. Segmentation analysis
6. Cohort trends
7. Executive summary with recommendations

### Phase 1: Data Quality Assessment

In [ ]:
# YOUR CODE HERE
# Perform comprehensive data quality checks:
# 1. Missing values by column
# 2. Data type validation
# 3. Duplicate detection
# 4. Logical inconsistencies (e.g., conversions > clicks)
# 5. Date range verification
# 6. Campaign completeness (do all campaigns have all days?)
#
# Create a data quality scorecard



### Phase 2: Statistical Summary by Segment

In [ ]:
# YOUR CODE HERE
# Create statistical summaries:
# 1. Overall portfolio statistics
# 2. Statistics by channel
# 3. Statistics by campaign type
# 4. Statistics by campaign
# 5. Weekly trend statistics
#
# Identify:
# - Best and worst performing segments
# - Most and least consistent segments
# - Segments with unusual patterns



### Phase 3: Distribution and Outlier Analysis

In [ ]:
# YOUR CODE HERE
# Analyze distributions:
# 1. Check distribution shape for key metrics (skewness, kurtosis)
# 2. Identify outliers using IQR method for all metrics
# 3. Investigate outlier days - what happened?
# 4. Determine if outliers are:
#    - Data quality issues → should be corrected
#    - Legitimate extreme events → should be noted but kept
#    - Anomalies → should be investigated
# 5. Create an outlier report with recommendations



### Phase 4: Correlation and Relationship Analysis

In [ ]:
# YOUR CODE HERE
# Correlation analysis:
# 1. Overall correlation matrix for key metrics
# 2. Correlations by channel (do relationships differ?)
# 3. Correlations by campaign type
# 4. Time-based correlations (early period vs late period)
# 5. Identify strongest predictors of:
#    - High ROAS
#    - Low CPA
#    - High conversion volume
#
# Insights: Which levers should we pull to improve performance?



### Phase 5: Campaign Segmentation

In [ ]:
# YOUR CODE HERE
# Segment campaigns into tiers:
# 
# Use campaign-level aggregates to create tiers based on:
# - ROAS (efficiency)
# - Conversion volume (scale)
# 
# Create 4 segments:
# 1. Stars: High ROAS + High volume → Scale these
# 2. Question Marks: Low ROAS + High volume → Optimize these
# 3. Cash Cows: High ROAS + Low volume → Maintain these
# 4. Dogs: Low ROAS + Low volume → Consider pausing
#
# Provide specific recommendations for each campaign



### Phase 6: Temporal Analysis

In [ ]:
# YOUR CODE HERE
# Time-based analysis:
# 1. Day-of-week patterns (are weekends different?)
# 2. Weekly trends (improving or declining?)
# 3. Campaign maturity analysis (performance in first 30 days vs later)
# 4. Seasonality or trend detection
# 5. Identify best and worst performing weeks
#
# Insights: 
# - When should we increase/decrease budgets?
# - How long does a campaign take to mature?
# - Are there seasonal patterns?



### Phase 7: Executive Summary

In [ ]:
# YOUR CODE HERE
# Create an executive summary including:
# 
# 1. Key Findings (3-5 bullets)
# 2. Data Quality Assessment (1-2 sentences)
# 3. Performance Highlights:
#    - Best performing campaigns/channels
#    - Worst performing campaigns/channels
# 4. Insights from Analysis:
#    - What drives success?
#    - What predicts failure?
#    - Surprising findings
# 5. Actionable Recommendations (5-7 specific actions)
# 6. Next Steps for deeper analysis
#
# Format this as a clear, executive-ready summary



### 🎓 Week 5 Complete!

**Congratulations!** You've completed Week 5 of the Marketing Measurement Partner Academy.

**What You've Mastered:**
- ✅ Descriptive statistics (central tendency, spread)
- ✅ Distribution analysis and outlier detection
- ✅ Correlation analysis and interpretation
- ✅ Data profiling and quality assessment
- ✅ Segmentation analysis
- ✅ Cohort analysis
- ✅ End-to-end exploratory data analysis

**Your EDA Toolkit:**
You can now:
- Quickly assess data quality
- Identify patterns, trends, and anomalies
- Segment data meaningfully
- Find correlations and relationships
- Generate data-driven hypotheses
- Communicate findings clearly

**Next Week Preview:**
Week 6 will cover **Data Visualization**:
- Matplotlib fundamentals
- Seaborn for statistical plots
- Time series visualization
- Distribution plots
- Heatmaps and correlation plots
- Dashboard design principles
- Marketing performance dashboard project

**You're becoming a data-driven marketing expert!** 🚀

---